In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from nbeats_keras.model import NBeatsNet as NBeatsKeras
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
#from nbeats_pytorch.model import NBeatsNet as NBeatsPytorch
from keras.optimizers import RMSprop, Adam
import time
from keras.models import load_model
#from target_data_electronic70_7 import target_X, target_y ,test_X, test_y
#from m4databasis21_7 import base_domain,zt_in,zt_out,M4Meta,inputsize,train_12,train_12_y
from sklearn.metrics import mean_squared_error,mean_absolute_error,mean_absolute_percentage_error
from tensorflow.keras.losses import Loss
import tensorflow as tf
#from m4databasis35_7_70_7 import train_35,train_35_y,train_70,train_70_y
from tensorflow.keras import layers
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Dense, LayerNormalization, MultiHeadAttention, Dropout, Add, Concatenate,Flatten,Reshape
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Lambda
from tensorflow.keras.callbacks import EarlyStopping
import time


2024-11-01 00:08:13.772935: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-11-01 00:08:13.951674: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcudart.so.11.0'; dlerror: libcudart.so.11.0: cannot open shared object file: No such file or directory
2024-11-01 00:08:13.951692: I tensorflow/compiler/xla/stream_executor/cuda/cudart_stub.cc:29] Ignore above cudart dlerror if you do not have a GPU set up on your machine.
2024-11-01 00:08:14.643826: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory
2024-

In [2]:

data = 'ele'
target_X= pd.read_csv(f"../data/{data}_train_input_7.csv").iloc[:,1:].values.astype(np.float32) /10000
target_y =pd.read_csv(f"../data/{data}_train_output_7.csv").iloc[:,1:].values.astype(np.float32) /10000

X_train = target_X[:-round(target_X.shape[0]*0.2),:].astype(np.float32)
y_train = target_y[:-round(target_y.shape[0]*0.2)].astype(np.float32)

X_train_val= target_X[-round(target_X.shape[0]*0.2):,:].astype(np.float32)
y_train_val =target_y[-round(target_y.shape[0]*0.2):].astype(np.float32)


test_X= pd.read_csv(f"../data/{data}_val_input_7.csv").iloc[:,1:].values.astype(np.float32)/10000
test_y =pd.read_csv(f"../data/{data}_val_output_7.csv").iloc[:,1:].values.astype(np.float32)/10000
#y_train.astype(np.float32)
backcast_length = X_train.shape[1]
forecast_length = y_train.shape[1]

In [3]:
#################################################################################
# loss SMAPE
class SMAPE(Loss):
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 예측 값의 차원을 맞춤
       # y_pred=tf.clip_by_value(y_pred, 1e-10, tf.reduce_max(y_pred))
       # y_true = tf.clip_by_value(y_true, 1e-10, tf.reduce_max(y_true))
        
        numerator = 100 * tf.abs(y_true- y_pred )
        denominator =  (tf.abs(y_true ) + tf.abs(y_pred))/2
        smape =  numerator /  denominator #tf.clip_by_value(denominator, 1e-10, tf.reduce_max(denominator))
        return tf.reduce_mean(smape)

#################################################################################
# loss MASE
class MASE(Loss):
    def __init__(self, training_data, period, *args, **kwargs):
        super().__init__(*args, **kwargs)
        self.scale = self.calculate_scale(training_data, period)
    def seasonal_diff(data, period):
        return data[period:] - data[:-period]

    def calculate_scale(self, training_data, period):
        # 주기 차분 계산
        diff = seasonal_diff(training_data, period)
        scale = np.mean(np.abs(diff))
        return scale
    
    def call(self, y_true, y_pred):
        y_pred = tf.reshape(y_pred, tf.shape(y_true))  # 차원 맞추기
        error = tf.abs(y_true - y_pred)
        return tf.reduce_mean(error / self.scale)

def seasonal_diff(data, period):
    return data[period:] - data[:-period]
#################################################################################
# 하이퍼파라미터 인자 설정
def hyperparameter():
    # 1 backcast
    # 2 forecast
    # 3 inputdim
    # 4 outputdim
    # 5 unit
    # 6 bacth size
    return X_train.shape[1],1,y_train.shape[1]

#################################################################################
# nbeats 모델 생성 함수
def build_model(input_timesteps,features,output_timesteps):
    model = Sequential()
    model.add(LSTM(128, return_sequences=True, input_shape=(input_timesteps, features)))
    model.add(LSTM(64, return_sequences=True))
    #model.add(LSTM(unit, return_sequences=True))
    # Use Lambda layer to select the last 'output_timesteps' outputs
    model.add(Lambda(lambda x: x[:, -24:, :]))
    model.add(TimeDistributed(Dense(1)))
    optimizer = Adam(learning_rate=0.001)
    model.compile(loss='mse', optimizer=optimizer)
    return model

#################################################################################
# 부트스트랩 샘플링
# 배깅

def train_bagging_models(num_models, loss_fn , epochs_, patience_,batch_size_,lr):
    models = {}
    input_timesteps,features,output_timesteps= hyperparameter()
    historys = []
    for n in range(num_models):
        K.clear_session()
        model = build_model(input_timesteps,features,output_timesteps)
       # model.set_weights(pretrained_weights)  # 전이 학습 가중치 적용
        optimizer = Adam(learning_rate=lr)
        model.compile(optimizer=optimizer , loss=loss_fn)
        
        # 부트스트랩 샘플링
        #select = np.random.choice(len(X_train), size=len(X_train), replace=True)
        #X_bootstrap = X_train[select]
        #y_bootstrap = y_train[select]
        early_stop = EarlyStopping(monitor='val_loss', patience = patience_, verbose = 1, restore_best_weights=True)
        history = model.fit(X_train, y_train, batch_size = batch_size_,
                  epochs=epochs_, verbose=1, 
                  callbacks=[early_stop],
                validation_data = [X_train_val,y_train_val])
        models[f'model_{n+1}'] = model
        historys.append(history)
        #models.append(model)
        print(f"'########################################################Model{n}")
    return models,historys

#################################################################################




#################################################################################

def bagging_predict(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return np.median(predictions, axis=0)

def bagging_predict2(models, X):
    predictions = np.array([model.predict(X) for model in models.values()])
    return predictions

In [4]:
model_num = 10

start_time = time.time()

mase_models = train_bagging_models(model_num, MASE(y_train,24),2000,10,8,0.001)
mape_models = train_bagging_models(model_num,'mape',2000,10,8,0.001)
smape_models = train_bagging_models(model_num, SMAPE(),2000,10,8,0.001)

end_time = time.time()
elapsed_time = end_time - start_time
#mae_models = train_bagging_models(model_num, 'mae',2000,10,8,0.001)
#mse_models = train_bagging_models(model_num, 'mse',2000,10,8,0.001)

2024-11-01 00:08:18.295553: E tensorflow/compiler/xla/stream_executor/cuda/cuda_driver.cc:267] failed call to cuInit: CUDA_ERROR_NO_DEVICE: no CUDA-capable device is detected
2024-11-01 00:08:18.295611: I tensorflow/compiler/xla/stream_executor/cuda/cuda_diagnostics.cc:156] kernel driver does not appear to be running on this host (ymlee2-desktop): /proc/driver/nvidia/version does not exist
2024-11-01 00:08:18.296254: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


Epoch 1/2000
73/73 [==============================] - 8s 89ms/step - loss: 1.4980 - val_loss: 0.9220
Epoch 2/2000
73/73 [==============================] - 6s 82ms/step - loss: 0.8605 - val_loss: 0.8039
Epoch 3/2000
73/73 [==============================] - 6s 83ms/step - loss: 0.6602 - val_loss: 0.5865
Epoch 4/2000
73/73 [==============================] - 6s 84ms/step - loss: 0.5728 - val_loss: 0.5020
Epoch 5/2000
73/73 [==============================] - 6s 85ms/step - loss: 0.5569 - val_loss: 0.5895
Epoch 6/2000
73/73 [==============================] - 6s 84ms/step - loss: 0.5724 - val_loss: 0.4767
Epoch 7/2000
73/73 [==============================] - 6s 84ms/step - loss: 0.5275 - val_loss: 0.5363
Epoch 8/2000
73/73 [==============================] - 6s 83ms/step - loss: 0.5170 - val_loss: 0.4741
Epoch 9/2000
73/73 [==============================] - 6s 84ms/step - loss: 0.4958 - val_loss: 0.4656
Epoch 10/2000
73/73 [==============================] - 6s 85ms/step - loss: 0.5087 - val_lo

In [5]:
elapsed_time

8033.056547641754

In [6]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models

smape_predictions = bagging_predict2(pred1, X_train_val)
mase_predictions =  bagging_predict2(pred2, X_train_val)
mape_predictions =  bagging_predict2(pred3, X_train_val)


5/5 [==============================] - 0s 32ms/step


In [7]:
pred1,_=smape_models
pred2,_=mase_models
pred3,_=mape_models

smape_predictions_G = bagging_predict2(pred1, test_X)
mase_predictions_G = bagging_predict2(pred2,test_X)
mape_predictions_G = bagging_predict2(pred3,test_X)
print('############################################################################################') 
print('############################################################################################') 


concat_G = np.concatenate([smape_predictions_G, mase_predictions_G,mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('original',mean_squared_error(test_y.flatten(),fin_pred_G.flatten()).round(5),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten()).round(5))
pd.DataFrame(fin_pred_G.flatten()).to_csv('lstm.csv')


concat_G = np.concatenate([mase_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mase',mean_squared_error(test_y.flatten(),fin_pred_G.flatten()).round(5),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten()).round(5))

concat_G = np.concatenate([mape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('mape',mean_squared_error(test_y.flatten(),fin_pred_G.flatten()).round(5),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten()).round(5))

concat_G = np.concatenate([smape_predictions_G],axis=0)
fin_pred_G = np.median(concat_G,axis=0)
#pd.DataFrame(fin_pred).to_csv("freezing_I.csv")
print('smape',mean_squared_error(test_y.flatten(),fin_pred_G.flatten()).round(5),mean_absolute_error(test_y.flatten(),fin_pred_G.flatten()).round(5))

print(np.mean([mean_squared_error(test_y.flatten(),mase_predictions_G[i].flatten()) for i in range(10)]))
print(np.mean([mean_absolute_error(test_y.flatten(),mase_predictions_G[i].flatten()) for i in range(10)]))

12/12 [==============================] - 0s 30ms/step
############################################################################################
############################################################################################
original 0.15862 0.25145
mase 0.15792 0.25723
mape 0.13123 0.23182
smape 0.27816 0.33717
0.20549028
0.29262868
